In [1]:
from ultralytics import YOLO

In [27]:
import torch

device = 'cpu'
# print('mps', torch.backends.mps.is_available())
if torch.cuda.is_available():
    device = '0'
device

mps True


'cpu'

In [60]:
model = YOLO("yolov8n-seg.pt")  # load a pretrained model
model.train(data="/Users/samedi/Documents/Coding/swans/data/swans.yaml", 
            epochs=50, 
            device=device, 
            imgsz=320,
            resume=False)
success = model.export(format="onnx")  # export the model to ONNX format
metrics = model.val()

New https://pypi.org/project/ultralytics/8.0.109 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.107 🚀 Python-3.10.6 torch-2.0.1 CPU
yolo/engine/trainer: task=segment, mode=train, model=/Users/samedi/Documents/Coding/swans/model/yolov8/config/colab25.pt, data=/Users/samedi/Documents/Coding/swans/model/yolov8/config/swans-seg.yaml, epochs=35, patience=50, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augme

KeyboardInterrupt: 

In [66]:
model = YOLO('yolov8s-cls.pt')
model.train(data="/Users/samedi/Documents/Coding/swans/data", 
            epochs=50, 
            device=device, 
            imgsz=320,
            resume=False)
success = model.export(format="onnx")  # export the model to ONNX format
metrics = model.val()

New https://pypi.org/project/ultralytics/8.0.109 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.107 🚀 Python-3.10.6 torch-2.0.1 CPU
yolo/engine/trainer: task=classify, mode=train, model=/Users/samedi/Documents/Coding/swans/model/yolov8/yolov8s-cls.pt, data=/Users/samedi/Documents/Coding/swans/data, epochs=100, patience=50, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, c

KeyboardInterrupt: 